<a href="https://colab.research.google.com/github/adityarao225/LanguageTranslation/blob/main/Untitled24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import json
import numpy as np
import pandas as pd
import random
import re
import tensorflow as tf
import unicodedata

from google.colab import files

In [2]:
TL = 'IT'
SL = 'EN'

df= pd.read_csv('/content/ita.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:SL, 1:TL})
df.iloc[1000,:]

EN          Tom lied.
IT    Tom ha mentito.
Name: 1000, dtype: object

In [3]:
# import pandas as pd

# # Assuming your CSV file is named 'data.csv'
# file_path = '/content/Dataset_English_Hindi.csv'

# # Using pandas to read the CSV file
# df = pd.read_csv(file_path)

# Accessing the columns and converting them to lists
data_target = df['EN'].tolist()
data_input = df['IT'].tolist()  # Adjust the column name as per your CSV

# Printing the first few elements of each list for verification
print("data_target:", data_target[:5])  # Adjust the slice to view more or fewer elements
print("data_input:", data_input[:5])    # Adjust the slice to view more or fewer elements


data_target: ['Hi.', 'Run!', 'Run!', 'Run!', 'Who?']
data_input: ['Ciao!', 'Corri!', 'Corra!', 'Correte!', 'Chi?']


In [4]:
# data[:3]

In [5]:
# SEPARATOR = ','
# data_input, data_target = map(list, zip(*[pair.split(SEPARATOR) for pair in data]))

In [6]:
print(data_input[56331])
print(data_target[56331])

Tu hai mentito a Tom?
Did you lie to Tom?


In [7]:
def preprocess_sentence(s):
  if not isinstance(s, str): # Check if s is a string
    s = str(s) # Convert to string if it's not
  # s = normalize_unicode(s)
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = s.strip()
  return s

In [8]:
# Preprocess both the source and target sentences.
data_preprocessed_input = [preprocess_sentence(s) for s in data_input]
data_preprocessed_target = [preprocess_sentence(s) for s in data_target]

In [9]:
data_preprocessed_input[:3]

['Ciao !', 'Corri !', 'Corra !']

In [10]:
from sklearn.model_selection import train_test_split


# Splitting into train (80%) and temporary (20%)
train_input, temp_input, train_target, temp_target = train_test_split(
    data_preprocessed_input, data_preprocessed_target, test_size=0.2, random_state=42)

# Splitting temporary into validation (50%) and test (50%)
validation_input, test_input, validation_target, test_target = train_test_split(
    temp_input, temp_target, test_size=0.5, random_state=42)

# Print the sizes of your sets
print("Train set size:", len(train_input))
print("Validation set size:", len(validation_input))
print("Test set size:", len(test_input))


Train set size: 273243
Validation set size: 34155
Test set size: 34156


In [11]:
def tag_target_sentences(sentences):
  tagged_sentences = map(lambda s: (' ').join(['<sos>', s, '<eos>']), sentences)
  return list(tagged_sentences)

In [12]:
train_tagged_preprocessed_target = tag_target_sentences(train_target)

In [13]:
train_tagged_preprocessed_target[:3]

['<sos> He loves soccer . <eos>',
 '<sos> I remained where I was . <eos>',
 '<sos> He is entirely in the wrong . <eos>']

In [14]:
val_tagged_preprocessed_target = tag_target_sentences(validation_target)

In [15]:
import re
import torch
from collections import Counter

class Tokenizer:
    def __init__(self, oov_token='<unk>', filters='"#$%&()*+-/:;=@[\\]^_`{|}~\t\n'):
        self.oov_token = oov_token
        self.filters = re.compile(f'[{re.escape(filters)}]')
        self.word_to_index = {}
        self.index_to_word = {}
        self.vocab_size = 0

    def fit_on_texts(self, texts, lowercase=False):
        # Tokenize and build vocabulary
        word_counts = Counter()
        for text in texts:
            tokens = self.tokenize(text, lowercase)
            word_counts.update(tokens)

        # Create word-to-index and index-to-word mappings
        self.word_to_index[self.oov_token] = 0
        self.index_to_word[0] = self.oov_token
        for idx, (word, _) in enumerate(word_counts.most_common(), start=1):
            self.word_to_index[word] = idx
            self.index_to_word[idx] = word

        self.vocab_size = len(self.word_to_index) + 1

    def tokenize(self, text, lowercase=False):
        # Tokenize and filter text
        text = re.sub(self.filters, ' ', text)
        if lowercase:
            text = text.lower()
        return text.split()

    def text_to_sequence(self, text, lowercase = False):
        # Convert text to a sequence of indices
        tokens = self.tokenize(text, lowercase )
        sequence = [self.word_to_index.get(token, 0) for token in tokens]
        return sequence

    def sequence_to_text(self, sequence):
        # Convert a sequence of indices to text
        tokens = [self.index_to_word.get(idx, self.oov_token) for idx in sequence]
        return ' '.join(tokens)




# # Example usage:
# train_preprocessed_input1 = ['This is a sample sentence.', 'Another example sentence.']
# tokenizer1 = Tokenizer()
# tokenizer1.fit_on_texts(train_preprocessed_input1)

# # Tokenize and convert text to sequences
# sequences = [tokenizer1.text_to_sequence(text) for text in train_preprocessed_input1]

# # Convert sequences back to text
# reconstructed_texts = [tokenizer1.sequence_to_text(sequence) for sequence in sequences]

# print("Vocabulary Size:", tokenizer1.vocab_size)
# print("Tokenized Sequences:", sequences)
# print("Reconstructed Texts:", reconstructed_texts)


In [16]:
source_tokenizer = Tokenizer()
source_tokenizer.fit_on_texts(train_input+validation_input)


In [17]:
source_tokenizer.vocab_size

31149

In [18]:
target_tokenizer = Tokenizer()

In [19]:
target_tokenizer.fit_on_texts(train_tagged_preprocessed_target+val_tagged_preprocessed_target,lowercase = True)

In [20]:
train_encoder_inputs = [source_tokenizer.text_to_sequence(text) for text in train_input]

In [21]:
train_encoder_inputs[:3]

[[3575, 13, 770, 1], [28, 2830, 5890, 1], [21, 10080, 121, 914, 1]]

In [22]:
# source_tokenizer.sequence_to_text(train_encoder_inputs[:3])
[source_tokenizer.sequence_to_text(text) for text in train_encoder_inputs[:3]]

['Ama il calcio .', "Sono restato dov'ero .", 'È interamente nel torto .']

In [23]:
def generate_decoder_inputs_targets(sentences, tokenizer):
  seqs = [tokenizer.text_to_sequence(text, lowercase = True) for text in sentences]
  decoder_inputs = [s[:-1] for s in seqs] # Drop the last token in the sentence.
  decoder_targets = [s[1:] for s in seqs] # Drop the first token in the sentence.

  return decoder_inputs, decoder_targets

In [24]:
train_decoder_inputs, train_decoder_targets = generate_decoder_inputs_targets(train_tagged_preprocessed_target, target_tokenizer)

In [25]:
print(train_decoder_inputs[0], train_decoder_targets[0])

print([target_tokenizer.sequence_to_text(x) for x in train_decoder_inputs[:1]],
      [target_tokenizer.sequence_to_text(y) for y in train_decoder_targets[:1]])

[1, 14, 613, 888, 3] [14, 613, 888, 3, 2]
['<sos> he loves soccer .'] ['he loves soccer . <eos>']


In [26]:
max_encoding_len = len(max(train_encoder_inputs, key=len))
max_encoding_len

100

In [27]:
max_decoding_len = len(max(train_decoder_inputs, key=len))
max_decoding_len

111

In [28]:
import numpy as np

# Assuming train_encoder_inputs, train_decoder_inputs, and train_decoder_targets are lists of sequences
# Each sequence is a list of integers

# Find the maximum sequence length
max_length_encoder = max(len(seq) for seq in train_encoder_inputs)
max_length_decoder = max(len(seq) for seq in train_decoder_inputs)

# Pad sequences for the encoder inputs
padded_train_encoder_inputs = [np.pad(seq, (0, max_length_encoder - len(seq)), mode='constant', constant_values=0) for seq in train_encoder_inputs]

# Pad sequences for the decoder inputs
padded_train_decoder_inputs = [np.pad(seq, (0, max_length_decoder - len(seq)), mode='constant', constant_values=0) for seq in train_decoder_inputs]

# Pad sequences for the decoder targets
padded_train_decoder_targets = [np.pad(seq, (0, max_length_decoder - len(seq)), mode='constant', constant_values=0) for seq in train_decoder_targets]


In [29]:
print(len(padded_train_encoder_inputs[0]))
print(padded_train_decoder_inputs[0])
print(padded_train_decoder_targets[0])

100
[  1  14 613 888   3   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0]
[ 14 613 888   3   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0]


In [30]:
[target_tokenizer.sequence_to_text([x]) for x in padded_train_decoder_inputs[0]]

['<sos>',
 'he',
 'loves',
 'soccer',
 '.',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk

In [31]:
def process_dataset(preprocessed_input,preprocessed_output):


  # Tag target sentences with <sos> and <eos> tokens.
  tagged_preprocessed_output = tag_target_sentences(preprocessed_output)
  tagged_preprocessed_input = tag_target_sentences(preprocessed_input)

  # Vectorize encoder source sentences.
  encoder_inputs = [source_tokenizer.text_to_sequence(preprocessed_input) for preprocessed_input in tagged_preprocessed_input]

  # Vectorize and create decoder input and target sentences.
  decoder_inputs, decoder_targets = generate_decoder_inputs_targets(tagged_preprocessed_output,
                                                                    target_tokenizer)


  # Pad sequences for the encoder inputs
  padded_encoder_inputs = [np.pad(seq, (0, max_length_encoder - len(seq)), mode='constant', constant_values=0) for seq in train_encoder_inputs]

  # Pad sequences for the decoder inputs
  padded_decoder_inputs = [np.pad(seq, (0, max_length_decoder - len(seq)), mode='constant', constant_values=0) for seq in train_decoder_inputs]

  # Pad sequences for the decoder targets
  padded_decoder_targets = [np.pad(seq, (0, max_length_decoder - len(seq)), mode='constant', constant_values=0) for seq in train_decoder_targets]


  # return padded_encoder_inputs, padded_decoder_inputs, padded_decoder_targets
  return torch.tensor(padded_encoder_inputs, dtype=torch.long), torch.tensor(padded_decoder_inputs, dtype=torch.long), torch.tensor(padded_decoder_targets, dtype=torch.long)




In [32]:
# # Process validation dataset

# # encoder_inputs = torch.randint(0, source_tokenizer.vocab_size, (32, max_encoding_len))
# # input_lengths = torch.tensor([len(seq) for seq in encoder_inputs])
# padded_val_encoder_inputs, padded_val_decoder_inputs, padded_val_decoder_targets = process_dataset(validation_input,validation_target)

In [33]:
# embedding_dim = 128
# hidden_dim = 256
# default_dropout=0.2
# batch_size = 32
# source_vocab_size= source_tokenizer.vocab_size

# encoder_inputs = torch.randint(0, source_vocab_size, (batch_size, max_encoding_len))
# input_lengths = torch.tensor([len(seq) for seq in encoder_inputs])
# epochs = 30

In [34]:
def train(model, dataloader, optimizer, criterion,max_examples):
    model.train()
    total_loss = 0
    examples_processed = 0
    predicted_sentences = []
    for batch in dataloader:
        if examples_processed >= max_examples:
            break
        src, tgt_input, tgt_output = batch
        src = src  # Ensure float32
        # print(src)
        tgt_input = tgt_input  # Ensure float32
        tgt_output = tgt_output  # Ensure float32


        src_mask = (src != 0).unsqueeze(-2)
        tgt_mask = (tgt_input != 0).unsqueeze(-2)
        tgt_mask = tgt_mask & (1 - torch.triu(torch.ones((1, tgt_input.size(1), tgt_input.size(1))), diagonal=1)).bool()

        optimizer.zero_grad()

        encoder_output = model.encode(src, src_mask)
        decoder_output = model.decode(encoder_output, src_mask, tgt_input, tgt_mask)
        output = model.projection_layer(decoder_output)

        # print(output)

        loss = criterion(output.view(-1, output.size(-1)), tgt_output.view(-1))
        loss.backward()

        optimizer.step()
            # Convert probabilities to predictions
            # predicted_indices = torch.argmax(output, dim=-1)

        # _, predicted_indices = torch.topk(output, k=1, dim=-1)
        # # predicted_indices = predicted_indices.squeeze(dim=-1)  # Remove the last dimension

        # # Convert indices to words using target_tokenizer.sequence_to_text (adjust as per your tokenizer)
        # predicted_sentences = []
        # for sentence_indices in predicted_indices:
        #     predicted_sentence = [target_tokenizer.sequence_to_text(indices) for indices in sentence_indices.tolist()]
        #     predicted_sentences.append(predicted_sentence)
        #     print("Predicted Sentence:", predicted_sentence)
        # # _, predicted_indices = torch.topk(output, k=2, dim=-1)
        # # predicted_indices = predicted_indices[:, :, 0]  # Selecting the second most probable index

        # #     # Decode predictions to words using target_tokenizer
        # # predicted_sentence = [target_tokenizer.sequence_to_text(x.squeeze()) for x in predicted_indices]

        # # predicted_sentences.append(predicted_sentence)

        # encoder_input_sentences = []
        # for i in range(src.size(0)):  # Iterate through batch dimension
        #     encoder_input_sentence = source_tokenizer.sequence_to_text(src[i].tolist())
        #     encoder_input_sentences.append(encoder_input_sentence)

        # # Print the predicted sentence and encoder input sentences
        # for encoder_input_sentence in encoder_input_sentences:
        #     print("Encoder Input Sentence:", encoder_input_sentence)
        #     # print("Predicted Sentence:", predicted_sentence)
        total_loss += loss.item()
        examples_processed += src.size(0)

    return total_loss / len(dataloader),output

def evaluate(model, dataloader, criterion, target_tokenizer, max_examples=10):
    model.eval()
    total_loss = 0
    examples_processed = 0
    # predicted_sentences = []
    main = []
    with torch.no_grad():
        for batch in dataloader:
            if examples_processed >= max_examples:
                break
            src, tgt_input, tgt_output = batch
            src = src # Ensure float32
            tgt_input = tgt_input # Ensure float32
            tgt_output = tgt_output  # Ensure float32

            # Print encoder input
            # print("Encoder Input (src):", src)
            assert src.size(0) == 1, "Batch size must be 1 for evaluation"

            src_mask = (src != 0).unsqueeze(-2)
            tgt_mask = (tgt_input != 0).unsqueeze(-2)
            tgt_mask = tgt_mask & (1 - torch.triu(torch.ones((1, tgt_input.size(1), tgt_input.size(1))), diagonal=1)).bool()

            encoder_output = model.encode(src, src_mask)
            decoder_output = model.decode(encoder_output, src_mask, tgt_input, tgt_mask)
            output = model.projection_layer(decoder_output)

            # Print decoder output
            # print("Decoder Output (before projection):", decoder_output)
            # print("Output (after projection):", output)

            loss = criterion(output.view(-1, output.size(-1)), tgt_output.view(-1))
            total_loss += loss.item()
            examples_processed += src.size(0)

            _, predicted_indices = torch.topk(output, k=1, dim=-1)
            # predicted_indices = predicted_indices.squeeze(dim=-1)  # Remove the last dimension

            # Convert indices to words using target_tokenizer.sequence_to_text (adjust as per your tokenizer)
            predicted_sentences = []
            for sentence_indices in predicted_indices:
                predicted_sentence = [target_tokenizer.sequence_to_text(indices) for indices in sentence_indices.tolist()]
                # predicted_sentence = [indices for indices in sentence_indices.tolist()]
                predicted_sentences.append(predicted_sentence)
                print("Predicted Sentence:", predicted_sentence)

            main.append(predicted_sentences)

            encoder_input_sentences = []
            for i in range(src.size(0)):  # Iterate through batch dimension
                encoder_input_sentence = source_tokenizer.sequence_to_text(src[i].tolist())
                encoder_input_sentences.append(encoder_input_sentence)

            # Print the predicted sentence and encoder input sentences
            for encoder_input_sentence in encoder_input_sentences:
                print("Encoder Input Sentence:", encoder_input_sentence)

    return total_loss / len(dataloader),main




In [35]:
import math

import torch
import torch.nn as nn



class InputEmbeddings(nn.Module):

    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        self.pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        # self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:,:x.shape[1],:].detach()
        return self.dropout(x)

class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter (nn.Parameter makes it learnable)
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        # x: (batch, seq_len, hidden_size)
         # Keep the dimension for broadcasting
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # Keep the dimension for broadcasting
        std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # eps is to prevent dividing by zero or when std is very small
        return self.alpha * (x - mean) / (std + self.eps) + self.bias


class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))


class ResidualConnection(nn.Module):

        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)

        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            mask = mask.unsqueeze(1)
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        return self.w_o(x)

class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return torch.log_softmax(self.proj(x), dim=-1)


class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # (batch, seq_len, d_model)
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # (batch, seq_len, d_model)
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x)

def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    # Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    # Create the encoder and decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))

    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)


    return transformer







In [36]:
from torch.utils.data import DataLoader, TensorDataset

# Update vocabulary sizes
src_vocab_size = source_tokenizer.vocab_size
tgt_vocab_size = target_tokenizer.vocab_size


# Process training and validation data
train_encoder_inputs, train_decoder_inputs, train_decoder_targets = process_dataset(
    train_input, train_target
)
val_encoder_inputs, val_decoder_inputs, val_decoder_targets = process_dataset(
    validation_input, validation_target
)

# Create DataLoader for training and validation
batch_size = 32
train_dataset = TensorDataset(torch.tensor(padded_train_encoder_inputs, dtype = torch.long), torch.tensor(padded_train_decoder_inputs, dtype =torch.long), torch.tensor(padded_train_decoder_targets,dtype=torch.long))
val_dataset = TensorDataset(val_encoder_inputs, val_decoder_inputs, val_decoder_targets)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1)

# Initialize model, optimizer, and loss function
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
src_vocab_size = source_tokenizer.vocab_size
tgt_vocab_size = target_tokenizer.vocab_size
model = build_transformer(src_vocab_size, tgt_vocab_size, max_encoding_len, max_decoding_len, d_model=512, N=6, h=8, dropout=0.1, d_ff=2048)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss(ignore_index=0)


<ipython-input-31-9bc25dd9341a>:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(padded_encoder_inputs, dtype=torch.long), torch.tensor(padded_decoder_inputs, dtype=torch.long), torch.tensor(padded_decoder_targets, dtype=torch.long)


In [37]:
[target_tokenizer.sequence_to_text([29749])]

['<unk>']

In [38]:
train_encoder_inputs[0]

tensor([3575,   13,  770,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])

In [39]:
# source_tokenizer.fit_on_texts(train_input)

In [40]:
[target_tokenizer.sequence_to_text([x]) for x in val_decoder_targets[1].tolist()]

['i',
 'remained',
 'where',
 'i',
 'was',
 '.',
 '<eos>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 

In [ ]:
# Training and validation loop
num_epochs = 1000
for epoch in range(num_epochs):
    train_loss,out = train(model, train_dataloader, optimizer, criterion,max_examples=500)
    val_loss, predicted_sentences = evaluate(model, val_dataloader, criterion,target_tokenizer,max_examples = 10)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')


Predicted Sentence: ['<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>'

In [ ]:
for x in predicted_sentences:
   print([target_tokenizer.sequence_to_text(y) for y in x])
   print("\n")

In [ ]:
print([target_tokenizer.sequence_to_text([6])])